In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import json
import sys

In [18]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 100.0.4896
[WDM] - Get LATEST chromedriver version for 100.0.4896 google-chrome
[WDM] - Driver [C:\Users\GuoshengCai\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [7]:
def find_department_pages(driver, departments_url, no_pages, css_selector, nth_element=None):
    driver.get(departments_url)
    items = []
    for i in range(no_pages):
        if i != 0:
            to_click = driver.find_element(By.LINK_TEXT, 'Next')
            to_click.click()
        elements = driver.find_elements(By.CSS_SELECTOR, css_selector)
        if nth_element is not None:
            elements = elements[nth_element].find_elements(By.CSS_SELECTOR,"*")
        for element in elements:
            try:
                tmp = element.find_element(By.TAG_NAME, 'a')
                item = {'department': tmp.get_attribute('textContent'), 'url': tmp.get_attribute('href')}
                items.append(item)
            except:
                pass
    return items

In [26]:
def find_faculty_page(driver, url):
    driver.get(url)
    elements = driver.find_elements(By.XPATH, 
    '//a[contains(@href,"faculty") or contains(@href,"profile") or contains(@href,"people") or contains(@href,"directory") or contains(@href,"staff")]')
    if len(elements) == 0:
        lk = None
    else:
        lk = select_lk(elements)
    return lk

In [27]:
def select_lk(elements):
    priority_dict = {'faculty directory': 7, 'faculty list': 6, 'faculty': 5, 'people': 4,'directory': 3,
                     'profiles': 2, 'staff': 1}
    lk = None
    priority = -1
    for element in elements:
        s = element.get_attribute("innerText").lower()
        curr_priority = 0
        for t in priority_dict:
            if t == s:
                curr_priority = priority_dict.get(t)
                break
        if curr_priority > priority:
            priority = curr_priority
            lk = element.get_attribute('href')
    return lk

# Columbia

In [24]:
departments_url = "https://www.columbia.edu/content/academics/departments"
no_pages = 5
css_selector = '.dynamic-grid-listing-item.grid-item.angular-animate.ng-trans.ng-trans-fade-down.ng-scope'
columbia_departments = find_department_pages(driver, departments_url, no_pages, css_selector)

In [91]:
for item in columbia_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page(driver, url)})

In [93]:
with open(sys.path[0] + '/../data/columbia/columbia_directories_updated.json', 'w') as f:
    json.dump(columbia_departments, f)

# Dartmouth

In [123]:
departments_url = "https://home.dartmouth.edu/academics/departments-programs"
no_pages = 1
nth_element = 2
css_selector = '.clearfix.text-formatted.field.field--name-body.field--type-text-with-summary.field--label-hidden.field__item'
dartmouth_departments = find_department_pages(driver, departments_url, no_pages, css_selector, nth_element)

In [131]:
# https://stackoverflow.com/questions/7867537/how-to-select-a-drop-down-menu-value-with-selenium-using-python
def find_faculty_page_dartmouth(driver, url):
    driver.get(url)
    try:
        element = driver.find_element(By.XPATH, 
        '//a[contains(@href,"people")]') 
        lk = element.get_attribute('href')
        driver.get(lk)
        # https://stackoverflow.com/questions/26958193/how-to-select-any-element-from-the-web-element-with-display-none-attribute-us
        driver.execute_script("document.getElementById('edit-field-people-local-tags-tid').style.display='inline-block';")
        element = WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.ID, 'edit-field-people-local-tags-tid')))
        select = Select(element)
        try:
            select.select_by_visible_text('Faculty')
        except:
            pass

        try:
            select.select_by_visible_text('faculty')
        except:
            select.select_by_index(1)

        to_click = driver.find_element(By.ID, 'edit-submit-people')
        to_click.click()
        currentURL = driver.current_url
        return currentURL
    except:
        return None

In [132]:
for item in dartmouth_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page_dartmouth(driver, url)})

In [134]:
with open(sys.path[0] + '/../data/dartmouth/dartmouth_directories.json', 'w') as f:
    json.dump(dartmouth_departments, f)

# Yale

In [26]:
departments_url = "https://www.yale.edu/academics/departments-programs"
no_pages = 1
css_selector = '.department_item_heading'
yale_departments = find_department_pages(driver, departments_url, no_pages, css_selector)

In [31]:
for item in yale_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page(driver, url)})

In [34]:
with open(sys.path[0] + '/../data/yale/yale_directories.json', 'w') as f:
    json.dump(yale_departments, f)

# Princeton

In [1]:
# only undergraduate departments
def find_department_pages_princeton_undergrad(driver):
    driver.get('https://admission.princeton.edu/academics/degrees-departments')
    elements = driver.find_elements(By.TAG_NAME, "p")
    items = []
    for element in elements:
        #element = element.find_element(By.XPATH, "..")
        try:
            tmp = element.find_element(By.TAG_NAME, 'a')
            item = {'department': tmp.get_attribute('textContent'), 'url': tmp.get_attribute('href')}
            items.append(item)
        except:
            pass
    return items

In [36]:
princeton_grad = find_department_pages_princeton_grad(driver)

In [4]:
princeton_departments = find_department_pages_princeton_undergrad(driver)

In [6]:
princeton_departments = princeton_departments[1:-1]

In [41]:
for item in princeton_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page(driver, url)})

In [42]:
with open(sys.path[0] + '/../data/princeton/princeton_directories.json', 'w') as f:
    json.dump(princeton_departments, f)

# Brown

In [8]:
departments_url = "https://www.brown.edu/academics/departments"
no_pages = 1
css_selector = '.news-items'
brown_departments = find_department_pages(driver, departments_url, no_pages, css_selector)

In [4]:
def find_faculty_page_brown(driver, url):
    driver.get(url)
    elements = driver.find_elements(By.XPATH, 
    '//a[contains(@href,"faculty") or contains(@href,"profile") or contains(@href,"people") or contains(@href,"directory") or contains(@href,"staff")]')
    if len(elements) == 0:
        lk = None
    else:
        lk = select_lk_brown(elements)
    return lk

In [11]:
def select_lk_brown(elements):
    priority_dict = {'faculty directory': 7, 'faculty list': 6, 'faculty': 5, 'people': 4,'directory': 3,
                     'profiles': 2, 'staff': 1}
    lk = None
    priority = -1
    for element in elements:
        if element.get_attribute('href') in ['https://www.brown.edu/gateway/faculty', 'http://www.brown.edu/gateway/faculty']:
            continue
        s = element.get_attribute("innerText").lower()
        curr_priority = 0
        for t in priority_dict:
            if t == s:
                curr_priority = priority_dict.get(t)
                break
        if curr_priority > priority:
            priority = curr_priority
            lk = element.get_attribute('href')
    return lk

In [12]:
for item in brown_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page_brown(driver, url)})

In [14]:
with open(sys.path[0] + '/../data/brown/brown_directories.json', 'w') as f:
    json.dump(brown_departments, f)

# Cornell

In [17]:
def find_department_pages_cornell(driver):
    driver.get('https://www.cornell.edu/academics/departments.cfm')
    items = []
    elements = driver.find_elements(By.TAG_NAME, 'li')
    for element in elements:
        try:
            tmp = element.find_element(By.TAG_NAME, 'a')
            item = {'department': tmp.get_attribute('textContent'), 'url': tmp.get_attribute('href')}
            items.append(item)
        except:
            pass
    return items

In [20]:
cornell_departments = find_department_pages_cornell(driver)

In [23]:
for i in range(len(cornell_departments)):
    print(i, cornell_departments[i].get('department'))

0 About Cornell
1 University Facts
2 Maps & Directions
3 Mission & Vision
4 Core Values
5 Locations
6 Ithaca
7 New York City
8 Diversity
9 Indigenous Engagement
10 Economic Impact
11 Sustainability
12 Global
13 Cornell Leadership
14 Admissions
15 Apply
16 Admissions
17 Dates and Deadlines
18 Costs and Aid 
19 Admissions
20 Fields of Study
21 Financial Support
22 Summer Sessions
23 Winter Sessions
24 Part-time Study
25 Academics
26 Fields of Study
27 Colleges & Schools
28 Departments
29 Library
30 Continuing Education
31 Office of the Provost
32 Academic Calendar
33 Course Catalog
34 Global Learning
35 Learning Goals
36 Research
37 
            Extreme Talent. Radical Collaboration. Results That Matter.
38 Vice President for Research and Innovation
39 Centers & Institutes
40 Technology Licensing (CTL)
41 Graduate Research
42 Undergraduate Research
43 OVPRI Research Publications
44 Public Engagement
45 
            
            CCE helps Buffalo get school food truck rolling
46 Cornell C

In [24]:
cornell_departments = cornell_departments[71:192]

In [28]:
for item in cornell_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page(driver, url)})

WebDriverException: Message: target frame detached
  (Session info: chrome=100.0.4896.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x009F7413+2389011]
	Ordinal0 [0x00989F61+1941345]
	Ordinal0 [0x0087C520+836896]
	Ordinal0 [0x0086D957+776535]
	Ordinal0 [0x0086C995+772501]
	Ordinal0 [0x0086CF98+774040]
	Ordinal0 [0x0086CF28+773928]
	Ordinal0 [0x00873144+799044]
	Ordinal0 [0x0086E16D+778605]
	Ordinal0 [0x0086E685+779909]
	Ordinal0 [0x0086E46F+779375]
	Ordinal0 [0x0086DA86+776838]
	Ordinal0 [0x0086C995+772501]
	Ordinal0 [0x0086CF98+774040]
	Ordinal0 [0x0088499D+870813]
	Ordinal0 [0x0087EB6B+846699]
	Ordinal0 [0x0087E6E5+845541]
	Ordinal0 [0x0087EEDF+847583]
	Ordinal0 [0x0087F1B0+848304]
	Ordinal0 [0x008A8EF2+1019634]
	Ordinal0 [0x008A949B+1021083]
	Ordinal0 [0x008D606C+1204332]
	Ordinal0 [0x008C4194+1130900]
	Ordinal0 [0x008D4302+1196802]
	Ordinal0 [0x008C3F66+1130342]
	Ordinal0 [0x0089E546+976198]
	Ordinal0 [0x0089F456+980054]
	GetHandleVerifier [0x00BA9632+1727522]
	GetHandleVerifier [0x00C5BA